<div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 30px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
  Deep Learning for NLP
  </div> 
  
<div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 30px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
  Part II - 2 <br><br><br>
  Machine Translation
  </div> 

  <div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 20px; 
      text-align: center; 
      padding: 15px;">
  </div> 

  <div style=" float:right; 
      font-size: 12px; 
      line-height: 12px; 
  padding: 10px 15px 8px;">
  Jean-baptiste AUJOGUE
  </div> 

### Part I

1. Word Embedding

2. Sentence Classification

3. Language Modeling

4. Sequence Labelling


### Part II

1. Text Classification

2. <font color=red>**Machine Translation**</font>


### Part III

8. Abstractive Summarization

9. Question Answering

10. Chatbot


</div>

***

<a id="plan"></a>

| | | | |
|------|------|------|------|
| **Content** | [Corpus](#corpus) | [Modules](#modules) | [Model](#model) | 



# Packages

In [50]:
import sys
import warnings
from __future__ import unicode_literals, print_function, division
import os
from io import open
import unicodedata
import string
import time
import math
import re
import random
import pickle
import copy
from unidecode import unidecode
import itertools
import matplotlib
import matplotlib.pyplot as plt


# for special math operation
from sklearn.preprocessing import normalize


# for manipulating data 
import numpy as np
#np.set_printoptions(threshold=np.nan)
import pandas as pd
import bcolz # see https://bcolz.readthedocs.io/en/latest/intro.html
import pickle


# for text processing
import gensim
from gensim.models import KeyedVectors
#import spacy
import nltk
#nltk.download()
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.stem.porter import PorterStemmer


# for deep learning
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")

warnings.filterwarnings("ignore")
print('python version :', sys.version)
print('pytorch version :', torch.__version__)
print('DL device :', device)

python version : 3.6.5 |Anaconda, Inc.| (default, Mar 29 2018, 13:32:41) [MSC v.1900 64 bit (AMD64)]
pytorch version : 1.3.1
DL device : cuda


In [2]:
path_to_NLP = 'C:\\Users\\Jb\\Desktop\\NLP'

In [3]:
sys.path.append(path_to_NLP + '\\libDL4NLP')

<a id="corpus"></a>

# Corpus

[Back to top](#plan)

Le texte est importé et mis sous forme de liste, où chaque élément représente un texte présenté sous forme d'une liste de mots.<br> Le corpus est donc une fois importé sous le forme :<br>

- corpus = [text]<br>
- text   = [word]<br>
- word   = str<br>

In [4]:
# --------------------------- Normalisation -------------------------------
def normalizeString(s):
    '''Remove rare symbols from a string'''
    def unicodeToAscii(s):
        """Turn a Unicode string to plain ASCII, thanks to http://stackoverflow.com/a/518232/2809427"""
        return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')
 
    s = unicodeToAscii(s.lower().strip())
    #s = re.sub(r"[^a-zA-Z0-9?&\%\-\_]+", r" ", s) 
    s = re.sub("\(", r" ( ", s)
    s = re.sub("\)", r" ) ", s)
    s = re.sub(r"\.", r" . ", s)
    s = re.sub(r",", r" , ", s)
    s = re.sub(r"!", r" ! ", s)
    s = re.sub(r":", r" : ", s)
    s = re.sub(r"-", r" - ", s)
    s = re.sub(r"'", r" ' ", s)
    s = re.sub(r";", r" ; ", s)
    s = re.sub(r' +', r' ', s).strip()
    return s 



#--------------------- import des dialogues --------------------
def importDialogues(path, limit = None):
    '''Import a textfile containing dialogues and returns a list, each element 
       corresponding to a dialogue and also being under the form of a list, with 
       each element being a list of two elements : an element giving a user 
       utterance and another element giving the bot response. Both elements are 
       normalized strings.
       Ex. The dialogue :
       
               hi    hello what can i help you with today
               can you book a table    i m on it
               
       now becomes :
       
              [['hi', 'hello what can i help you with today'], 
               ['can you book a table', 'i m on it']]
               
       Lines corresponding to user utterance with no bot response are discarted.
    '''
    def cleanS(s):
        cleans = normalizeString(s)
        cleans = cleans.replace('?', ' ? ').strip()
        return cleans
    
    dialogues = []
    dialogues_import = open(path, encoding='utf-8').read().strip().split('\n\n')
    for i, d in enumerate(dialogues_import):
        dialogue = []
        lines = d.split('\n')
        for l in lines:
            if len(l.split('\t')) == 2 :
                pair = [cleanS(s) for s in l.split('\t')]
                dialogue.append(pair)
            elif len(l.split('\t')) == 3 :
                pair = [cleanS(s) for s in l.split('\t')[:2]]
                dialogue.append(pair)
        dialogues.append(dialogue)
        if limit is not None and i == limit -1 : break
    return dialogues


def getUniqueQAs(dialogues) :
    uniq = []
    for qa in dialogues :
        if qa not in uniq : uniq.append(qa)
    return uniq



#------------------ Dictionnaire des mots variables -----------------------------
def motVar(file):
    '''Applies to the Master's program dataset.
       Import the collection of pairs token-content for a set of variable words.
    '''
    lines = open(file, encoding='utf-8').read().strip().split('\n')
    motsVar = {}
    for l in lines :
        cle, valeur = l.split('\t')
        motsVar[cle.lower()] = valeur
    return motsVar

In [85]:
motsVar    = motVar(path_to_NLP + '\\data\\M2DS\\M2DS_2019_07\\chatbot-M2-DS-Variables.txt')
dialogues  = importDialogues(path_to_NLP + '\\data\\M2DS\\M2DS_2019_10\\ChatbotDS_P_Train.tsv')
dialogues_tst = importDialogues(path_to_NLP + '\\data\\M2DS\\M2DS_2019_10\\ChatbotDS_P_Test.tsv')

In [86]:
qa_trn = [[qa[0], qa[1] + ' EOS'] for dialogue in dialogues for qa in dialogue] # getUniqueQAs(
qa_tst = [[qa[0], qa[1] + ' EOS'] for dialogue in dialogues_tst for qa in dialogue] # getUniqueQAs(

print(len(qa_trn), len(qa_tst))

200224 20561


In [87]:
for qa in dialogues[0] : print(qa[0], '\t', qa[1])

il vaut mieux suivre quel parcours  ? 	 les deux parcours sont equivalents , c ' est a vous de faire un choix .
quand est ce que commence les cours  ? 	 la rentree est le daterentree
comment je prend contact avec la scolarite  ? 	 les contacts de la scolarite sont accessibles sur : sitescolarite
est ce que il y a des cours en anglais  ? 	 plusieurs supports de cours sont en anglais
tous les cours sont obligatoires  ? 	 tous les cours sont obligatoires , il n ' existe pas d ' options .
merci pour les renseignements 	 je vous en prie
peux tu m ' aider  ? 	 je suis la pour vous renseigner sur le master 2 data science .
le cours de data visualisation est - il en ligne  ? 	 vous devriez prendre contact avec l ' enseignant de l ' ue et lui demander
j ' ai envie de faire de la data science 	 alors vous devriez songer a candidater a ce master !
il y a des livres a lire en statistique  ? 	 vous devriez prendre contact avec l ' enseignant de l ' ue et lui demander
il y a combien de candidats  ? 

<a id="modules"></a>

# 1 Modules

### 1.1 Word Embedding module

[Back to top](#plan)

All details on Word Embedding modules and their pre-training are found in **Part I - 1**. We consider here a FastText model trained following the Skip-Gram training objective.

In [8]:
from libDL4NLP.models.Word_Embedding import Word2Vec as myWord2Vec
from libDL4NLP.models.Word_Embedding import Word2VecConnector
from libDL4NLP.utils.Lang import Lang

In [9]:
from gensim.models.fasttext import FastText
from gensim.test.utils import datapath, get_tmpfile

In [90]:
corpus_in  = [['SOS'] + [w for w in qa[0].split(' ')] + ['EOS'] for qa in qa_trn]
corpus_out = [['SOS'] + [w for w in qa[1].split(' ')]           for qa in qa_trn]

In [98]:
def prepareWord2vec(corpus) :
    fastText_word2vec = FastText(size = 75, 
                                 window = 5, 
                                 min_count = 1, 
                                 negative = 20,
                                 sg = 1)
    fastText_word2vec.build_vocab(corpus)
    print(len(fastText_word2vec.wv.vocab))
    fastText_word2vec.train(sentences = corpus, 
                            epochs = 5,
                            total_examples = fastText_word2vec.corpus_count)
    word2vec = Word2VecConnector(fastText_word2vec)
    return word2vec

In [99]:
word2vec_in  = prepareWord2vec(corpus_in)
word2vec_out = prepareWord2vec(corpus_out)

782
896


In [100]:
word2vec_in.word2vec.most_similar('bonjou')

[('bonjour', 0.9852568507194519),
 ('mal', 0.8179829120635986),
 ('principal', 0.6931003928184509),
 ('principalement', 0.6789224147796631),
 ('bien', 0.6547073125839233),
 ('english', 0.6544337272644043),
 ('handle', 0.6447221040725708),
 ('ok', 0.6249094605445862),
 ('cordialement', 0.6209567785263062),
 ('speak', 0.618739128112793)]

In [101]:
word2vec_out.word2vec.most_similar('bonjou')

[('bonjour', 0.9725614786148071),
 ('accord', 0.786864161491394),
 ('comment', 0.6106002330780029),
 ('aider', 0.6066528558731079),
 ('accordez', 0.5910225510597229),
 ('excusez', 0.5597412586212158),
 ('monotonie', 0.5362932682037354),
 ('puis', 0.5213144421577454),
 ('vais', 0.5169289112091064),
 ('dynamiques', 0.4986083209514618)]

### 1.2 Contextualization module

[Back to top](#plan)

This module consists of a bi-directional _Gated Recurrent Unit_ (GRU) that supports packed sentences :

In [15]:
from libDL4NLP.modules import RecurrentEncoder

### 1.3 Attention module

[Back to top](#plan)

<a id="attention"></a>

We use here a classical Attention Module :

In [16]:
from libDL4NLP.modules import Attention

### 1.4 Decoder module

[Back to top](#plan)

<a id="decoder"></a>

#### 1.4.1 Classical Decoder Module

In [17]:
#from libDL4NLP.modules import Decoder

In [18]:
class Decoder(nn.Module):
    '''Transforms a vector into a sequence of words'''
    def __init__(self, word2vec, hidden_dim, 
                 n_layers = 1,
                 dropout = 0.1,
                 bound = 25
                ):
        super(Decoder, self).__init__()
        # relevant quantities
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.bound = bound
        # modules
        self.word2vec = word2vec
        self.gru = nn.GRU(word2vec.output_dim, 
                          hidden_dim, 
                          n_layers, 
                          dropout = dropout, 
                          batch_first = True)
        self.out = nn.Linear(hidden_dim, word2vec.lang.n_words)
        self.act = F.log_softmax
        self.dropout = nn.Dropout(dropout)
        
    def generateWord(self, hidden, word_index):
        # update hidden state
        embedding = self.word2vec.embedding(word_index) # size (batch_size, 1, embedding_dim)
        embedding = self.dropout(embedding)
        _, hidden = self.gru(embedding, hidden)         # size (n_layers, batch_size, embedding_dim)
        # generate next word
        log_prob = self.out(hidden[-1])                 # size (batch_size, lang_size)
        log_prob = self.act(log_prob, dim = 1)          # size (batch_size, lang_size)
        return log_prob, hidden
    
    def forward(self, hidden, device = None) :
        answer = []
        word_index = self.word2vec.lang.getIndex('SOS')
        EOS_token  = self.word2vec.lang.getIndex('EOS')
        for t in range(self.bound) :
            # compute next word
            word_index = Variable(torch.LongTensor([[word_index]]))    # size (1, 1)
            if device is not None : word_index = word_index.to(device) # size (1, 1)
            log_prob, hidden = self.generateWord(hidden, word_index)
            word_index = log_prob.topk(1)[1].item()
            # add to output
            if word_index == EOS_token : break
            else : answer.append(word_index)
        return answer

#### 1.4.2 Attention Decoder Module

In [19]:
#from libDL4NLP.modules import AttnDecoder

<a id="model"></a>

# 2 Machine Translation Model

[Back to top](#plan)


In [43]:
class EncoderDecoder(nn.Module) :
    def __init__(self, device, tokenizer, word2vec_in, word2vec_out, 
                 hidden_dim = 100,
                 n_layers = 1, 
                 bound = 25,
                 dropout = 0, 
                 optimizer = optim.SGD
                 ):
        super(EncoderDecoder, self).__init__()
        
        # embedding
        self.tokenizer    = tokenizer
        self.word2vec_in  = word2vec_in
        self.word2vec_out = word2vec_out
        self.context      = RecurrentEncoder(word2vec_in.output_dim, hidden_dim, n_layers, dropout, bidirectional = True)
        self.decoder      = Decoder(word2vec_out, hidden_dim, n_layers, dropout, bound)
        
        # optimizer
        self.ignore_index_in  = self.word2vec_in.lang.getIndex('PADDING_WORD')
        self.ignore_index_out = self.word2vec_out.lang.getIndex('PADDING_WORD')
        self.criterion = nn.NLLLoss(size_average = False, ignore_index = self.ignore_index_out)
        self.optimizer = optimizer
        
        # load to device
        self.device = device
        self.to(device)
        
    def nbParametres(self) :
        return sum([p.data.nelement() for p in self.parameters() if p.requires_grad == True])
    
    def forward(self, sentence, color_code = '\033[94m'):
        # encode sentence
        words  = self.tokenizer(sentence)
        embeddings = self.word2vec_in(words, self.device)
        _, hidden  = self.context(embeddings)
        # sum along directions
        if self.context.bidirectional :
            hidden = hidden.view(self.context.n_layers, 2, -1, self.context.hidden_dim)
            hidden = torch.sum(hidden, dim = 1) # size (n_layers, batch_size, hidden_dim)
        # generate answer
        answer = self.decoder(hidden, self.device)
        answer = [self.word2vec_out.lang.index2word[i] for i in answer]
        print(' '.join(words + [':', color_code] + answer + ['\033[0m']))
        return
    
    def generatePackedSentences(self, sentences, batch_size = 32) : 
        sentences.sort(key = lambda s: len(s[1]), reverse = True)
        packed_data = []
        for i in range(0, len(sentences), batch_size) :
            # prepare input and target pack
            pack = sentences[i:i + batch_size]
            pack.sort(key = lambda s: len(self.tokenizer(s[0])), reverse = True)
            pack0 = [[self.word2vec_in.lang.getIndex(w) for w in self.tokenizer(qa[0])] for qa in pack]
            pack0 = [[w for w in words if w is not None] for words in pack0]
            pack1 = [[self.word2vec_out.lang.getIndex(w) for w in self.tokenizer(qa[1])] for qa in pack]
            pack1 = [[w for w in words if w is not None] for words in pack1]
            lengths = torch.tensor([len(p) for p in pack0])           # size (batch_size) 
            # padd packs
            pack0 = list(itertools.zip_longest(*pack0, fillvalue = self.ignore_index_in))
            pack0 = Variable(torch.LongTensor(pack0).transpose(0, 1)) # size (batch_size, max_length0) 
            pack1 = list(itertools.zip_longest(*pack1, fillvalue = self.ignore_index_out))
            pack1 = Variable(torch.LongTensor(pack1))       # WARNING : size (max_length1, batch_size) 
            packed_data.append([[pack0, lengths], pack1])
        return packed_data
    
    def compute_accuracy(self, sentences) :
        batches = self.generatePackedSentences(sentences, batch_size = 32)
        score = 0
        for batch, target in batches :
            embeddings  = self.word2vec_in.embedding(batch[0].to(self.device))
            hiddens, _  = self.context(embeddings, lengths = batch[1].to(self.device))
            attended, _ = self.attention(hiddens)
            if self.bin_mode : 
                vects  = self.out(attended).view(-1)
                target = target.to(self.device).view(-1)
                score += sum(torch.abs(target - self.act(vects)) < 0.5).item()
            else : 
                log_probs = F.log_softmax(self.out(attended.squeeze(1)))
                target    = target.to(self.device).view(-1)
                score    += sum([target[i].item() == log_probs[i].data.topk(1)[1].item() for i in range(target.size(0))])
        return score * 100 / len(sentences)
    
    def fit(self, batches, iters = None, epochs = None, tf_ratio = 0, lr = 0.025, random_state = 42,
              print_every = 10, compute_accuracy = True):
        """Performs training over a given dataset and along a specified amount of loops"""
        def asMinutes(s):
            m = math.floor(s / 60)
            s -= m * 60
            return '%dm %ds' % (m, s)

        def timeSince(since, percent):
            now = time.time()
            s = now - since
            rs = s/percent - s
            return '%s (- %s)' % (asMinutes(s), asMinutes(rs))
        
        def computeSuccess(log_probs, targets) :
            success = sum([self.ignore_index_out != targets[i].item() == log_probs[i].topk(1)[1].item() \
                           for i in range(targets.size(0))])
            return success
        
        def computeLogProbs(batch, target, tf_ratio = 0, compute_accuracy = True) :
            loss = 0
            success = 0
            forcing = (random.random() < tf_ratio)
            # encode sentences
            embeddings = self.word2vec_in.embedding(batch[0].to(self.device))
            _, hidden  = self.context(embeddings, lengths = batch[1].to(self.device)) # size (n_layers * num_directions, batch_size, hidden_dim)
            # sum along directions
            if self.context.bidirectional :
                hidden = hidden.view(self.context.n_layers, 2, -1, self.context.hidden_dim)
                hidden = torch.sum(hidden, dim = 1)               # size (n_layers, batch_size, hidden_dim)
            # compute answers
            word_index = self.word2vec_out.lang.getIndex('SOS')
            word_index = Variable(torch.LongTensor([word_index])) # size (1)
            word_index = word_index.expand(target.size(1))        # size (batch_size)
            for t in range(target.size(0)) :
                # compute word probs
                log_prob, hidden = self.decoder.generateWord(hidden, word_index.unsqueeze(1).to(self.device))
                # compute loss
                loss += self.criterion(log_prob, target[t])
                if compute_accuracy : success += computeSuccess(log_prob, target[t])
                # apply teacher forcing
                if forcing : word_index = target[t]                             # size (batch_size) 
                else       : word_index = log_prob.topk(1, dim = 1)[1].view(-1) # size (batch_size)
            return loss, success       

        def printScores(start, iter, iters, tot_loss, tot_loss_words, print_every, compute_accuracy) :
            avg_loss = tot_loss / print_every
            avg_loss_words = tot_loss_words / print_every
            if compute_accuracy : print(timeSince(start, iter / iters) + ' ({} {}%) loss : {:.3f}  accuracy : {:.1f} %'.format(iter, int(iter / iters * 100), avg_loss, avg_loss_words))
            else                : print(timeSince(start, iter / iters) + ' ({} {}%) loss : {:.3f}                     '.format(iter, int(iter / iters * 100), avg_loss))
            return 0, 0

        def trainLoop(batch, optimizer, tf_ratio = 0, compute_accuracy = True):
            """Performs a training loop, with forward pass, backward pass and weight update."""
            optimizer.zero_grad()
            self.zero_grad()
            target = batch[1].to(self.device)
            total = np.sum(target.data.cpu().numpy() != self.ignore_index_out)
            loss, success = computeLogProbs(batch[0], target, tf_ratio, compute_accuracy)
            loss.backward()
            optimizer.step()
            return float(loss.item() / total), float(success * 100 / total)
        
        # --- main ---
        self.train()
        np.random.seed(random_state)
        start = time.time()
        optimizer = self.optimizer([param for param in self.parameters() if param.requires_grad == True], lr = lr)
        tot_loss = 0  
        tot_acc  = 0
        if epochs is None :
            for iter in range(1, iters + 1):
                batch = random.choice(batches)
                loss, acc = trainLoop(batch, optimizer, tf_ratio, compute_accuracy)
                tot_loss += loss
                tot_acc += acc      
                if iter % print_every == 0 : 
                    tot_loss, tot_acc = printScores(start, iter, iters, tot_loss, tot_acc, print_every, compute_accuracy)
        else :
            iter = 0
            iters = len(batches) * epochs
            for epoch in range(1, epochs + 1):
                print('epoch ' + str(epoch))
                np.random.shuffle(batches)
                for batch in batches :
                    loss, acc = trainLoop(batch, optimizer, tf_ratio, compute_accuracy)
                    tot_loss += loss
                    tot_acc += acc 
                    iter += 1
                    if iter % print_every == 0 : 
                        tot_loss, tot_acc = printScores(start, iter, iters, tot_loss, tot_acc, print_every, compute_accuracy)
        return

#### Training

In [102]:
max([len(qa[1].split(' ')) for qa in qa_trn])

70

In [108]:
chatbot = EncoderDecoder(device = torch.device("cpu"),
                         tokenizer = lambda s : normalizeString(s).split(' '),
                         word2vec_in = word2vec_in,
                         word2vec_out = word2vec_out,
                         hidden_dim = 75, 
                         n_layers = 2,
                         bound = 75,
                         dropout = 0.1,
                         optimizer = optim.SGD)

chatbot.nbParametres()

307122

In [104]:
chatbot

EncoderDecoder(
  (word2vec_in): Word2VecConnector(
    (twin): Word2Vec(
      (embedding): Embedding(784, 75)
    )
    (embedding): Embedding(784, 75)
  )
  (word2vec_out): Word2VecConnector(
    (twin): Word2Vec(
      (embedding): Embedding(898, 75)
    )
    (embedding): Embedding(898, 75)
  )
  (context): RecurrentEncoder(
    (dropout): Dropout(p=0.1, inplace=False)
    (bigru): GRU(75, 75, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)
  )
  (decoder): Decoder(
    (word2vec): Word2VecConnector(
      (twin): Word2Vec(
        (embedding): Embedding(898, 75)
      )
      (embedding): Embedding(898, 75)
    )
    (gru): GRU(75, 75, num_layers=2, batch_first=True, dropout=0.1)
    (out): Linear(in_features=75, out_features=897, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (criterion): NLLLoss()
)

In [105]:
torch.cuda.empty_cache()

In [106]:
batches = chatbot.generatePackedSentences(qa_trn, batch_size = 32)
len(batches)

6257

In [ ]:
chatbot.fit(batches, epochs = 1, lr = 0.001, tf_ratio = 1,  print_every = 100)
chatbot.fit(batches, epochs = 1, lr = 0.001, tf_ratio = 0.5,  print_every = 100)
chatbot.fit(batches, epochs = 1, lr = 0.00025, tf_ratio = 0.5,  print_every = 100)

epoch 1
0m 15s (- 16m 10s) (100 1%) loss : 5.452  accuracy : 7.6 %
0m 31s (- 15m 56s) (200 3%) loss : 4.851  accuracy : 10.5 %
0m 47s (- 15m 48s) (300 4%) loss : 4.522  accuracy : 17.4 %
1m 5s (- 15m 51s) (400 6%) loss : 4.159  accuracy : 23.2 %
1m 21s (- 15m 43s) (500 7%) loss : 3.634  accuracy : 31.9 %
1m 38s (- 15m 29s) (600 9%) loss : 3.188  accuracy : 38.8 %
1m 54s (- 15m 7s) (700 11%) loss : 2.760  accuracy : 46.6 %
2m 10s (- 14m 53s) (800 12%) loss : 2.659  accuracy : 48.6 %
2m 28s (- 14m 41s) (900 14%) loss : 2.467  accuracy : 51.9 %
2m 44s (- 14m 25s) (1000 15%) loss : 2.025  accuracy : 60.0 %
3m 1s (- 14m 11s) (1100 17%) loss : 2.145  accuracy : 56.5 %
3m 18s (- 13m 55s) (1200 19%) loss : 2.209  accuracy : 55.0 %
3m 32s (- 13m 32s) (1300 20%) loss : 2.048  accuracy : 60.5 %
3m 49s (- 13m 16s) (1400 22%) loss : 1.854  accuracy : 61.4 %
4m 6s (- 13m 1s) (1500 23%) loss : 1.937  accuracy : 59.4 %
4m 21s (- 12m 40s) (1600 25%) loss : 1.573  accuracy : 67.0 %
4m 37s (- 12m 23s) (1

2m 21s (- 11m 3s) (1100 17%) loss : 0.827  accuracy : 84.2 %
2m 34s (- 10m 51s) (1200 19%) loss : 0.750  accuracy : 85.8 %


In [60]:
# save
#torch.save(denoiser.state_dict(), path_to_NLP + '\\saves\\models\\DL4NLP_I4a_sentence_denoiser_2.pth')

# load
#denoiser.load_state_dict(torch.load(path_to_NLP + '\\saves\\models\\DL4NLP_I4a_sentence_denoiser.pth'))

#### Evaluation

In [80]:
chatbot.eval()
chatbot('salut ca va ?')

salut ca va ? :  la page ecandidat est disponible sur le lien suivant : siteecandidat 
